<a href="https://colab.research.google.com/github/HemanthKumarBodduboina/AI-Projects/blob/main/Resume_Analyzer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [15]:
import os
from google.colab import userdata

os.environ["OPENAI_API_KEY"] = userdata.get('AIproject')

In [16]:
from google.colab import files

# Upload PDF (it will ask you to choose a file)
uploaded = files.upload()

Saving resume.pdf to resume (1).pdf


In [17]:
!pip install pymupdf

In [18]:
import fitz  # PyMuPDF

# Open the uploaded resume
pdf_path = "resume.pdf"
doc = fitz.open(pdf_path)

resume_text = ""
for page in doc:
    resume_text += page.get_text("text")

print("Extracted Resume Text:\n")
print(resume_text[:1000])  # print first 1000 chars

Extracted Resume Text:

Bodduboina Hemanth kumar
Email-id : hemanthkumarbodduboina@gmail.com
Mobile No.: 7013438637
https://github.com/HemanthKumarBodduboina
ACADEMIC DETAILS
Year
Degree/Exam
Institute
GPA/Marks(%)
Sep, 2021 - July, 2025
B.TECH in ECE
Graphic Era University Dehradun
8.06%10
2021
BIEAP
Sri chaitanya Jr college
95.02 %
2019
BSEAP
Penna English Medium High school
98.05 %
PROJECTS
• Image Classification : Developed an artificial neural network classifier for recognizing handwritten digits
which was trained and tested using MNIST dataset giving 98% accuracy.
• Spam Detection : Developed a SVM classifier for recognizing if an email is spam or not. handwritten digits
which was trained and tested using MNIST dataset giving 98% accuracy.
• CNN : Image classification of dogs and cats using Convolutional Neural networks : FCNN, Convolutional
Layers , Pooling layers, Deep Learning .
• Project on speech recognition through emotions : Developed an AI-based speech recognition system 

In [19]:
import openai

# openai.api_key = "AIproject" # This line is not needed if using the client method

job_desc = """
We are hiring an AI Engineer skilled in NLP, Deep Learning, and MLOps (AWS, Docker, Kubernetes).
"""

prompt = f"""
You are a career advisor.
Compare this resume with the job description.
Return:
1. Match Score (0-100)
2. Missing Skills
3. Resume Improvements

Resume: {resume_text}
Job: {job_desc}
"""

# Create an OpenAI client instance
client = openai.OpenAI()


response = client.chat.completions.create(
    model="gpt-3.5-turbo",
    messages=[{"role": "user", "content": prompt}]
)

print(response.choices[0].message.content)

Match Score: 60

Missing Skills:
- MLOps experience (AWS, Docker, Kubernetes)
- Specific NLP projects or experience
- Specific Deep Learning projects or experience

Resume Improvements:
1. Add a section highlighting experience with MLOps tools like AWS, Docker, and Kubernetes
2. Include specific NLP and Deep Learning projects or experiences to showcase expertise in those areas
3. Consider obtaining certifications or completing projects related to MLOps, NLP, and Deep Learning to strengthen your profile
